In [97]:
import pandas as pd
import numpy as np

items = pd.read_csv("Items.csv",nrows=2000)
sales = pd.read_csv("Sales.csv",nrows=2000)
promotions = pd.read_csv("Promotion.csv",nrows=2000)
supermarkets = pd.read_csv("Supermarkets.csv",nrows=2000)

In [98]:
# Step 1: Data Cleaning & Standardization
def clean_dataframe(df):
    df = df.drop_duplicates()  # Remove duplicates
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)  # Trim whitespace
    return df

# Apply cleaning
items = clean_dataframe(items)
sales = clean_dataframe(sales)
promotions = clean_dataframe(promotions)
supermarkets = clean_dataframe(supermarkets)

C:\Users\prasa\AppData\Local\Temp\ipykernel_11780\410444424.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)  # Trim whitespace


In [66]:
supermarkets.head(3)

,supermarket_No,postal-code
0,199,30319
1,200,30134
2,201,30066


In [99]:
sales.rename(columns={"supermarket": "Supermarket_No"}, inplace=True)
promotions.rename(columns={"supermarkets": "Supermarket_No"}, inplace=True)
supermarkets.rename(columns={"supermarket_No": "Supermarket_No"}, inplace=True)


# Handled missing values
items.fillna({'Description': 'Unknown', 'Brand': 'Unknown', 'Size': 'Unknown'}, inplace=True)
sales.fillna({'Voucher': 0, 'CustomerId': 'Guest'}, inplace=True)
promotions.fillna({'feature': 'None', 'display': 'None'}, inplace=True)

sales['Time of transactions'] = pd.to_datetime(sales['time'])
sales['Day'] = pd.to_datetime(sales['day'])
sales['Amount'] = sales['amount'].astype(float)
sales['Units'] = sales['units'].astype(int)

In [56]:
supermarkets.head(3)

,supermarket_No,postal-code
0,199,30319
1,200,30134
2,201,30066


In [100]:
# Step 2: Merge DataFrames
sales = sales.merge(items, on="code", how="left")
sales = sales.merge(supermarkets, on="Supermarket_No", how="left")
sales = sales.merge(promotions, on=["code", "Supermarket_No", "province"], how="left")

# Step 3: Feature Engineering
sales["Weekday"] = sales["Time of transactions"].dt.day_name()
sales["Month"] = sales["Time of transactions"].dt.month
sales["IsWeekend"] = sales["Weekday"].isin(["Saturday", "Sunday"]).astype(int)

In [69]:
sales.head(3)

,code,amount,units,time,province,week_x,customerId,Supermarket_No,basket,day,...,type,brand,size,postal-code,week_y,feature,display,Weekday,Month,IsWeekend
0,7680850106,0.80,1,1100,2,1,125434,244,1,1,...,Type 2,Barilla,16 OZ,40222,NaN,NaN,NaN,Thursday,1,0
1,3620000470,3.59,1,1100,2,1,125434,244,1,1,...,Type 3,Bertolli,24 OZ,40222,NaN,NaN,NaN,Thursday,1,0
2,1800028064,2.25,1,1137,2,1,108320,244,2,1,...,Type 1,Hungry Jack,,40222,NaN,NaN,NaN,Thursday,1,0


In [101]:

# Basket-level analysis
sales["Total Basket Value"] = sales.groupby("basket")["Amount"].transform("sum")
sales["Average Units Per Basket"] = sales.groupby("basket")["units"].transform("mean")

# Promotion effectiveness
sales["Promotion Active"] = sales["feature"].notna().astype(int)
sales["Display Impact"] = sales["display"].notna().astype(int)

# Customer segmentation (High vs. Low spenders)
customer_spend = sales.groupby("customerId")["Amount"].sum().reset_index()
customer_spend["Customer Category"] = pd.qcut(customer_spend["Amount"], q=3, labels=["Low", "Medium", "High"])
sales = sales.merge(customer_spend, on="customerId", how="left")

# Step 4: Save the transformed data
sales.to_csv("cleaned_sales_data.csv", index=False)
sales.to_parquet("cleaned_sales_data.parquet", index=False)

print("Data cleaning and transformation complete! 🚀")


Data cleaning and transformation complete! 🚀


In [102]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Filter for promotion period (if you have a way to identify it, e.g., 'Feature' == 'Promo')
merged_df = pd.read_csv("cleaned_sales_data.csv")
promo_data = merged_df[merged_df["feature"] != "None"]
promo_data.head(3)

,code,amount,units,time,province,week_x,customerId,Supermarket_No,basket,day,...,display,Weekday,Month,IsWeekend,Total Basket Value,Average Units Per Basket,Promotion Active,Display Impact,Amount_y,Customer Category
0,7680850106,0.80,1,1100,2,1,125434,244,1,1,...,NaN,Thursday,1,0,4.39,1.0,0,0,4.39,High
1,3620000470,3.59,1,1100,2,1,125434,244,1,1,...,NaN,Thursday,1,0,4.39,1.0,0,0,4.39,High
2,1800028064,2.25,1,1137,2,1,108320,244,2,1,...,NaN,Thursday,1,0,2.25,1.0,0,0,2.25,Medium


In [107]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

merged_df = pd.read_csv("cleaned_sales_data.csv")
promo_data = merged_df[merged_df["feature"] != "None"]
print(promo_data.head(3))

         code  amount  units  time  province  week_x  customerId  \
0  7680850106    0.80      1  1100         2       1      125434   
1  3620000470    3.59      1  1100         2       1      125434   
2  1800028064    2.25      1  1137         2       1      108320   

   Supermarket_No  basket  day  ...  display   Weekday Month  IsWeekend  \
0             244       1    1  ...      NaN  Thursday     1          0   
1             244       1    1  ...      NaN  Thursday     1          0   
2             244       2    1  ...      NaN  Thursday     1          0   

   Total Basket Value Average Units Per Basket Promotion Active  \
0                4.39                      1.0                0   
1                4.39                      1.0                0   
2                2.25                      1.0                0   

  Display Impact Amount_y  Customer Category  
0              0     4.39               High  
1              0     4.39               High  
2              0

### Forecasting sales for specific items during promotional periods

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

merged_df = pd.read_csv("cleaned_sales_data.csv")
# promo_data = merged_df[merged_df["feature"] != "NaN"]

# Select features and target
X = promo_data[["code", "Supermarket_No",  "Month", "Weekday", "type", "brand","Promotion Active"]]
y = promo_data["amount"]  # or "units"

# One-hot encode categorical variables
X = pd.get_dummies(X, drop_first=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"RMSE for Forecasting Sales: {rmse}")


RMSE for Forecasting Sales: 12.862675440790383


### Analyzing the impact of promotional features using Random Forest


In [110]:
# promo_data = merged_df[merged_df["Feature"] != "None"]

X = promo_data[["code", "Supermarket_No",  "Month", "Weekday", "type", "brand","Promotion Active"]]
y = promo_data["amount"]

X = pd.get_dummies(X, drop_first=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

feature_importance = model.feature_importances_
importance_df = pd.DataFrame({
    "Feature": X.columns,
    "Importance": feature_importance
}).sort_values(by="Importance", ascending=False)

print("Feature Importance Analysis:")
print(importance_df)


Feature Importance Analysis:
                        Feature  Importance
1                Supermarket_No    0.781708
36          brand_Private Label    0.058645
0                          code    0.053560
4                   type_Type 2    0.049104
37  brand_Private Label Premium    0.018645
23                 brand_Hunt's    0.015762
6                   type_Type 4    0.006027
9                brand_Bertolli    0.002626
19               brand_Emeril's    0.002205
13               brand_Classico    0.002179
8                 brand_Barilla    0.001987
5                   type_Type 3    0.001511
39                   brand_Ragu    0.001247
32                brand_Pennant    0.000878
10               brand_Bisquick    0.000791
11                 brand_Cary's    0.000599
35                  brand_Prego    0.000483
27         brand_Maggi Spaetzle    0.000389
12             brand_Chef Pizza    0.000277
40            brand_San Giorgio    0.000229
30               brand_Newman's    0.000175
16 

### Predicting high-performing supermarkets based on transaction data.

In [113]:
from sklearn.ensemble import RandomForestClassifier

supermarket_sales = merged_df.groupby("Supermarket_No")["amount"].sum().reset_index()

threshold = supermarket_sales["amount"].quantile(0.75)
supermarket_sales["High_Performing"] = supermarket_sales["amount"].apply(lambda x: 1 if x > threshold else 0)

merged_supermarket_data = pd.merge(merged_df, supermarket_sales[["Supermarket_No", "High_Performing"]], on="Supermarket_No", how="left")

X = merged_supermarket_data[[ "Month", "Weekday", "code", "Promotion Active", "Display Impact"]]
y = merged_supermarket_data["High_Performing"]

X = pd.get_dummies(X, drop_first=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = (y_pred == y_test).mean()

print(f"Accuracy for Predicting High-Performing Supermarkets: {accuracy}")


Accuracy for Predicting High-Performing Supermarkets: 0.565


In [114]:
from sklearn.metrics import classification_report, accuracy_score

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy for Predicting High-Performing Supermarkets: {accuracy}")

report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)


Accuracy for Predicting High-Performing Supermarkets: 0.565
Classification Report:
              precision    recall  f1-score   support

           0       0.60      0.79      0.68       234
           1       0.45      0.24      0.31       166

    accuracy                           0.56       400
   macro avg       0.53      0.52      0.50       400
weighted avg       0.54      0.56      0.53       400

